# Tanzania Tourism Prediction

The Tanzanian tourism sector plays a significant role in the Tanzanian economy, contributing about 17% to the country’s GDP and 25% of all foreign exchange revenues. The sector, which provides direct employment for more than 600,000 people and up to 2 million people indirectly, generated approximately $2.4 billion in 2018 according to government statistics. Tanzania received a record 1.1 million international visitor arrivals in 2014, mostly from Europe, the US and Africa.
</br>
</br>
The objective of this notebook is to develop a machine learning model to predict what a tourist will spend when visiting Tanzania.The model can be used by different tour operators and the Tanzania Tourism Board to automatically help tourists across the world estimate their expenditure before visiting Tanzania.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor 
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, VotingRegressor, StackingRegressor
from xgboost import XGBRegressor

from sklearn.model_selection import RandomizedSearchCV

from scipy.stats import zscore


import warnings
warnings.filterwarnings('ignore')

# set the max columns to none
pd.set_option('display.max_columns', None)

# set printoptions not to display numbers in scientific notation
np.set_printoptions(suppress=True)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# set dark background for plots to be better seen in VSCode dark mode
from matplotlib import style
style.use('dark_background')


In [ ]:
# Import tanzania tourism data
raw_data_df = pd.read_csv('data/original_zindi_data/Train.csv')

# save currency conversion rate for conversion of TZS to EUR
TZS_RATE = 2628.57

# Split features X and target y

In [ ]:
X = raw_data_df.drop("total_cost", axis=1)
y = raw_data_df["total_cost"]

# Splitting data into train and test dataset

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

print(X_train.shape)
print(X_test.shape)

# First overview of the train data

For a first overview of the data, the features and the target value of the training data will be examined together in a DataFrame tourism_df.

In [ ]:
# Concatenate train features and target value for EDA
tourism_df = pd.concat([X_train,y_train], axis=1)

In [ ]:
tourism_df.head(10)

In [ ]:
tourism_df.shape

In [ ]:
tourism_df.info()

There are NaNs in the columns travel_with, total_female, total_male, and most_impressing.
Most features are categorical.

In [ ]:
tourism_df.describe()

In [ ]:
# comparing with mean of target value in test data
y_test.mean()

# Quick overview of the distribution of the target value Total Cost of trip

In [ ]:
# Create bins for target total_cost
total_cost_bins = np.arange(tourism_df.total_cost.min(), tourism_df.total_cost.max(), 1000000)
total_cost_bins_series = pd.cut(tourism_df.total_cost, bins=total_cost_bins, labels=total_cost_bins[:-1])	
total_cost_bins_series.name = 'total_cost_bins'

In [ ]:
# plot the target value distribution for train data
fig, axes = plt.subplots(1,1, figsize=(30, 10))
axes.tick_params(axis='x', rotation=90)
fig = sns.countplot(x=total_cost_bins_series)

fig.set_ylabel("Number of trips", fontsize = 20)
fig.set_xlabel("Total Cost of trip in TZS", fontsize = 20)

plt.savefig('images/total_cost_distribution.png', bbox_inches='tight')
plt.show()

The distribution of the target value for total cost of a trip is right skewed. Log transformation will be used on the target data to overcome this issue.

# Further examination of the categorical features and their influence on the total cost of a trip

In [ ]:
def plot_feature_to_total_cost(feature_column, label_name):
    age_group_df = tourism_df.groupby(feature_column).mean()['total_cost'].reset_index().merge(tourism_df.groupby(feature_column).count()['total_cost'], on=feature_column, suffixes=('_mean', '_count'))
    age_group_df.rename(columns={'total_cost_mean': 'mean_total_cost', 'total_cost_count': 'number_trips'}, inplace=True)

    #fig, axes = plt.subplots(1,1, figsize=(15, 8))
    plt.figure(figsize=(20,10))
    fig = sns.scatterplot(x=feature_column, y="mean_total_cost", size="number_trips",
                sizes=(40, 600), data=age_group_df)

    plt.ticklabel_format(style='plain', axis='y')
    fig.set_ylabel("Total Cost of trip in TZS", fontsize = 20)
    fig.set_xlabel(label_name, fontsize = 20)
    fig.set_ylabel(label_name, fontsize = 20)
    fig.legend_.set_title('Number of trips')
    plt.legend(loc=2, prop={'size': 16})
    plt.yticks(fontsize=12)
    plt.xticks(fontsize=12)
    
    plt.savefig(f'images/{feature_column}_total_cost.png', bbox_inches='tight')
    plt.show()

## Influence of Age Group on Total Cost of trip

In [ ]:
# Relation of total cost to age_group
plot_feature_to_total_cost('age_group', 'Age Group')


The higher the age, the higher the total cost of the trip.

# Influence of Purpose of travelling on Total Cost of trip

In [ ]:
# Relation of total cost to purpose
plot_feature_to_total_cost('purpose', 'Purpose of trip')

People travelling for the purpose of Leisure and Holidays spend significantly more money than people travelling for other reasons.

# Influence of Main Activity during travelling on Total Cost of trip

In [ ]:
# Relation of total cost to main_activity
plot_feature_to_total_cost('main_activity', 'Main Activity of tourists')

Total cost depends on main activity of the trip. The most money is spent when main activities are Diving and Sport Fishing, least for Hunting tourism and Mountain climbing.

# Influence of the people who are travelling with the examined tourist on Total Cost of trip

In [ ]:
# Relation of total cost to travel_with
plot_feature_to_total_cost('travel_with', 'People who accompany tourists on their trip')

Total cost depends on who is travelled with. The most money is spent when travelling with spouse and Children.

# Influence of Payment Mode on Total Cost of trip

In [ ]:
# Relation of total cost to payment_mode
plot_feature_to_total_cost('payment_mode', 'Payment Mode for trip')

There are differences among the 4 groups of payment methods in amount of total cost. People paying with Traveller Cheques seem to spend the most money.

# Influence of Tour Arrangement on Total Cost of trip

In [ ]:
# Relation of total cost to tour_arrangement
plot_feature_to_total_cost('tour_arrangement', 'Tour Arrangement')

Significantly more money is spent when travelling with a Package Tour than when travelling Independent.

# Deeper examination of Tour Arrangement distribution

In [ ]:
# Distribution of Tour arrangements
tour_arrangement_count = tourism_df['tour_arrangement'].value_counts()
tour_arrangement_count = tour_arrangement_count.reset_index()


labels = list(tour_arrangement_count['index'])
explode = (0, 0.1)  # only "explode" the 2nd slice (i.e. 'Hogs')

fig, ax = plt.subplots(1,1, figsize=(10, 8))
patches, texts, autotexts  = ax.pie(tour_arrangement_count['tour_arrangement'], explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90, textprops={'color':"black"})
ax.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
[text.set_color('white') for text in texts]
[autotext.set_color('black') for autotext in autotexts]

plt.savefig('images/tour_arrangement_percentage.png')
plt.show()

Distribution between Independant travelling and Package Tour is almost equal, less Package Tours than Independent travelling.

# Influence of origin Country on Total Cost of trip

In [ ]:
# Relation of total cost to country
country_df = tourism_df.groupby(['country']).mean()['total_cost'].reset_index()

fig, axes = plt.subplots(1,1, figsize=(30, 10))
fig = sns.barplot(x='country', y='total_cost', data=country_df, order = country_df.sort_values('total_cost', ascending = False).country)
plt.ticklabel_format(style='plain', axis='y')
axes.tick_params(axis='x', rotation=90)

fig.set_ylabel("Total Cost of trip in TZS", fontsize = 20)
fig.set_xlabel("Country where tourist is coming from", fontsize = 20)

plt.savefig('images/country_total_cost.png')
plt.show()

By far the most total cost of trips is observed for people coming from Dominica and Costarica, followed by Mexico, Slovenia and Russia.

# Number of trips per origin Country

In [ ]:
# plot the distribution of countries
fig, axes = plt.subplots(1,1, figsize=(30, 10))
axes.tick_params(axis='x', rotation=90)

fig = sns.countplot(x = tourism_df.country,
              order = tourism_df.country.value_counts().index)

fig.set_ylabel("Number of trips", fontsize = 20)
fig.set_xlabel("Country where tourist is coming from", fontsize = 20)

plt.savefig('images/country_distribution.png')
plt.show()


Most people going on a trip to Tantania come from the USA, UK and Italy, followed by France and Zimbabwe.

## Examination of numerical features

In [ ]:
# Calculate number of people and number of nights total
num_cols_tourism = list(tourism_df.columns[tourism_df.dtypes!=object])
tourism_num_cols_df = tourism_df[num_cols_tourism]
tourism_num_cols_df.eval('total_people = total_female + total_male', inplace=True)
tourism_num_cols_df.eval('total_nights = night_mainland + night_zanzibar', inplace=True)
tourism_num_cols_df

In [ ]:
# overview of all numerical features and the target variable
# temporarily remove outlier before plotting
outlier_index = tourism_num_cols_df.query('total_female > 40').index

sns.pairplot(tourism_num_cols_df.drop(outlier_index, axis=0)[['total_female', 'total_male','total_people', 'night_mainland', 'night_zanzibar', 'total_nights', 'total_cost']])
plt.ticklabel_format(style='plain', axis='y')
plt.savefig('images/pairplot_numerical_features.png')
plt.show()


There is no remarkable relation visible between the numerical columns.

In [ ]:
# Checking if there is at least one person travelling
tourism_df.query('total_female + total_male ==0')

This is strange, because there should be always at least one person travelling.
Those rows will still be kept in the dataset. Besides looking at these two columns compared, the data seems to be alright. The values of those two columns won't matter that much since they are also valid when looking at them separately.

# Data cleaning and feature engineering

From now on, the feature train dataset X_train will be treated.

In [ ]:
# Examine missing value in column most_impressing
X_train.most_impressing.value_counts()

This feature can be imputed with the value "No comments" sine this is a valid and appropriate value for missing information on this feature.

In [ ]:
# Examine missing value in column travel_with
print(X_train.travel_with.value_counts())

# percentage of missing values
print(f'{(X_train.travel_with.isnull().sum() / X_train.shape[0] * 100).round(2)} percent of values missing')


Since there is 23% of data missing for this feature, it is best to be imputed by a new category "missing".

## Handling Outliers in the numerical features

In [ ]:
# Find outliers with z-score
df_zscore_total_female  = zscore(X_train.total_female.fillna(X_train.total_female.median()))
df_zscore_total_male  = zscore(X_train.total_male.fillna(X_train.total_male.median()))
df_zscore_night_mainland  = zscore(X_train.night_mainland.fillna(X_train.night_mainland.median()))
df_zscore_night_zanzibar  = zscore(X_train.night_zanzibar.fillna(X_train.night_zanzibar.median()))

df_scores = pd.concat([df_zscore_total_female, df_zscore_total_male, df_zscore_night_mainland, df_zscore_night_zanzibar], axis=1)

# Calculate data loss based on 4 standard deviations of these 4 numeric features
loss_percentage = df_scores.query('total_female > 4 or total_male > 4 or night_mainland > 4 or night_zanzibar > 4').count()['total_female'] / X_train.shape[0] * 100
loss_rows = df_scores.query('total_female > 4 or total_male > 4 or night_mainland > 4 or night_zanzibar > 4')['total_female']

print(f'Percentage of data loss of the training data based on 4 standard deviations for the numeric features: {loss_percentage.round(2)}%')
print(f'Number of rows to be removed: {loss_rows.count().round(3)}')

All outliers based on a Z-Score of 4 for the numerical features will be removed from the dataset.

In [ ]:
# Remove outliers from train data
print(loss_rows.index)
X_train.drop(loss_rows.index, axis=0, inplace=True)
y_train.drop(loss_rows.index, axis=0, inplace=True)

# Handling 7 Package-columns

There are 7 boolean features which are dependent on the outcome of the feature 'tour_arrangement'. 
</br>


# Version 1: Impute package columns based on tour_arrangement

There are boolean columns that are only relevant for Package Tours, but they are also filled with 'No' for the Independent travellers. This might disturb the model!

In [ ]:
all_columns = list(X_train.columns)

# get list of all columns that only concern Package Tours
package_columns = [col for col in all_columns if 'package' in col]

package_columns
#X_train[X_train['tour_arrangement'] == "Independent"].index

X_train_imputed_package = X_train.copy()
X_test_imputed_package = X_test.copy()
# Replace values of package columns with 'Irrelevant' vor Independent travellers
X_train_imputed_package.loc[X_train['tour_arrangement'] == 'Independent', package_columns] = 'Irrelevant'
X_test_imputed_package.loc[X_test['tour_arrangement'] == 'Independent', package_columns] = 'Irrelevant'

#TODO: This needs to go into the Pipeline!!


# Version 2: Drop package columns version

In [ ]:
# Try a version without package columns
X_train_without_package_cols = X_train.drop(package_columns, axis=1)
X_test_without_package_cols = X_test.drop(package_columns, axis=1)

In [ ]:
X_test_without_package_cols

## First Simple Baseline Model

Because of a significant difference of mean total_cost within the categories of age_group, purpose, main_activity, tour_arrangement and payment_mode, it is assumed that the total_cost can be predicted by taking the mean total_cost grouped by these 5 categorical features.
Some combinations of the categories of these 5 features can be found in the test data but do not exist in the train data. Therefore, to fill in the missing mean values based on 5 features, the mean of total_cost grouped only by 2 features, age_group and tour_arrangement, is calculated.

In [ ]:
# Concatenate 5 base features and target value
basemodel_data_train = pd.concat([X_train[['age_group','purpose','main_activity','tour_arrangement','payment_mode']],y_train], axis=1)

# group data by base features and calculate mean of total_cost
basemodel_train = basemodel_data_train.groupby(['age_group','purpose','main_activity','tour_arrangement','payment_mode']).mean()['total_cost'].reset_index()

# Take easier model with only two features to impute missing values of first prediction where all 5 features in combination do not match test data
# Concatenate 2 base features and target value
basemodel_data_train_2features = pd.concat([X_train[['age_group','tour_arrangement']],y_train], axis=1)

# group data by base features and calculate mean of total_cost
basemodel_train_2_features = basemodel_data_train_2features.groupby(['age_group','tour_arrangement']).mean()['total_cost'].reset_index()

In [ ]:
# Make baseline predictions for train dataset

# Concatenate base features and target value
basemodel_data_train = X_train[['age_group','purpose','main_activity','tour_arrangement','payment_mode']]

# join basemodel_train to train data
predictions_train = basemodel_data_train.merge(basemodel_train, on=['age_group','purpose','main_activity','tour_arrangement','payment_mode'], suffixes=('_test', '_train'), how='left')

y_pred_train_basemodel = predictions_train['total_cost']


In [ ]:
# Make baseline predictions for test dataset

# Concatenate base features and target value
basemodel_data_test = pd.concat([X_test[['age_group','purpose','main_activity','tour_arrangement','payment_mode']],y_test], axis=1)

# join basemodel_train to test data
predictions_5_features = basemodel_data_test.merge(basemodel_train, on=['age_group','purpose','main_activity','tour_arrangement','payment_mode'], suffixes=('_actual', '_5features'), how='left')

# join basemodel_train_2features to predictions to fill the gaps
predictions_5_2_features = predictions_5_features.merge(basemodel_train_2_features, on=['age_group','tour_arrangement'], how='left')
predictions_5_2_features.rename(columns={'total_cost': 'total_cost_2features'}, inplace=True)


# for final prediction, take prediction with 5 features and fill NaNs with prediction for 2 features
predictions_5_2_features['total_cost_final_pred'] = predictions_5_2_features['total_cost_5features']
predictions_5_2_features['total_cost_final_pred'] = predictions_5_2_features['total_cost_final_pred'].fillna(value = predictions_5_2_features['total_cost_2features'])

y_pred_test_basemodel = predictions_5_2_features['total_cost_final_pred']
y_act_test_basemodel = predictions_5_2_features['total_cost_actual']

Remark: The total_cost are in TZS (Tansania-Schilling). The conversion rate is as follows: 1 EUR = 2628.57 TZN
</br>
RMSE is chosen as metric because it is best for the customer to see the amount of error directly in the common currency. Furthermore, because it is a squared metric, higher errors have a higher influence on the metric.

In [ ]:
def calculate_metrics(y_true, y_pred, print_metrics=True, calculate_r2=False):
    if calculate_r2:
        # R2 can only be used to assess train data
        r2 = r2_score(y_true, y_pred)
        if print_metrics:
            print(f'R2: {r2}')
        
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    mae = mean_absolute_error(y_true, y_pred)
    metrics = {'rmse' : rmse,
                'mae' : mae}

    if print_metrics:
        print(f'RMSE in TZS: {rmse}')
        print(f'RMSE in EUR: {rmse / TZS_RATE}')

        print(f'MAE in TZS: {mae}')
        print(f'MAE in EUR: {mae / TZS_RATE}')

    return metrics


In [ ]:
# Metrics for test data
calculate_metrics(y_act_test_basemodel,y_pred_test_basemodel, True)

In [ ]:
# Metrics for train data
calculate_metrics(y_train, y_pred_train_basemodel, True, True)

In [ ]:
y_test.median()

Conclusion: The RMSE of the baseline prediction is just a little bit better than the median.

Error Analysis of basemodel

In [ ]:
def plot_act_vs_pred(y_true, y_pred, image_name):
    fig, axes = plt.subplots(1,1, figsize=(15, 10))
    fig = sns.scatterplot(y_pred, y_true)

    # plot line for ideal prediction
    x = np.linspace(0,max(y_pred),5000000)
    y = x
    plt.plot(x, y, '-r', label='y=x')

    plt.ticklabel_format(style='plain', axis='y')
    plt.ticklabel_format(style='plain', axis='x')
    fig.set_ylabel("Actual values", fontsize = 20)
    fig.set_xlabel("Predicted values", fontsize = 20)
    plt.yticks(range(0, int(max(y_true)), 10000000))
    plt.xticks(range(0, int(max(y_pred)), 5000000))

    plt.savefig(f'images/{image_name}.png')
    plt.plot()

In [ ]:
# Plot actual vs. predicted value for the baseline model
plot_act_vs_pred(y_test, y_pred_test_basemodel, 'basemodel_act_vs_pred_values')

The red line shows the ideal prediction where the actual total cost would be the predicted total cost
</br>
The baseline model overestimates lower actual total costs and underestimates higher total costs.

In [ ]:
def residual_plot(y_test, y_pred_test, image_name, y_train=None, y_pred_train=None):
    # convert lists to arrays
    y_test = np.array(y_test)
    y_pred_test = np.array(y_pred_test)

    # Calculate difference of actual total_cost and predicted total_cost for test data
    y_pred_diff = y_pred_test - y_test

    # plot actual total_cost vs. difference between actual and predicted value
    fig, axes = plt.subplots(1,1, figsize=(15, 10))

    if y_train is None:
        fig = sns.scatterplot(x=y_pred_test, y=y_pred_diff)
        plt.xticks(range(-10000000, int(max(y_pred_test)), 10000000))
    else:
        y_train = np.array(y_train)
        y_pred_train = np.array(y_pred_train)

        num_test = len(y_test)
        num_train = len(y_train)
        y_test_total = np.concatenate((y_train, y_test))
        y_pred_total = np.concatenate((y_pred_train, y_pred_test))
        y_pred_diff_total = y_pred_total - y_test_total
        train_test_num = ['train data' for i in range(num_train)] + ['test data' for i in range(num_test)]

        fig = sns.scatterplot(x=y_pred_total, y=y_pred_diff_total, hue=train_test_num, alpha=0.7, palette=["yellow", "blue"])
        plt.xticks(range(-10000000, max(int(max(y_pred_train)), int(max(y_pred_test))), 10000000))

    plt.axhline(0, c='red')

    plt.ticklabel_format(style='plain', axis='y')
    plt.ticklabel_format(style='plain', axis='x')
    fig.set_xlabel("Predicted values", fontsize = 20)
    fig.set_ylabel("Standardized Residual", fontsize = 20)

    plt.yticks(range(-80000000, int(max(y_pred_diff)), 10000000))
    
    plt.savefig(f'images/{image_name}.png')
    plt.plot()


In [ ]:
# Plot the relation of actual value and the difference between actual and predicted values for the baseline model
residual_plot(y_test, y_pred_test_basemodel,'basemodel_residual_plot', y_train=y_train, y_pred_train=y_pred_train_basemodel)

In average, for lower total_cost, the prediction is too high and for higher total_cost it is too low. 
</br>
The error seems to be linear.

## Log transform target data

In [ ]:
log_transformer = FunctionTransformer(
        func=np.log1p,
        inverse_func=np.expm1
    )


y_train_log = log_transformer.transform(y_train)
y_test_log = log_transformer.transform(y_test)

## Building a Pipeline

In [ ]:
# Create feature lists for different kinds of pipelines

impute_median_features = ['total_female', 'total_male']      # num_features
impute_missing_features = ['travel_with']                    # cat_feature
impute_no_comments_features = ['most_impressing']            # cat_feature

# ID is a unique identifier for each tourist and therefore not relevant for the model
drop_features = ['ID']                                      # cat_feature

num_features = list(X_train.columns[X_train.dtypes!=object])
# remove items that also need to go through imputation
num_features = [x for x in num_features if x not in impute_median_features]

cat_features = list(X_train.columns[X_train.dtypes==object])
# remove items that also need to go through imputation or need to be dropped
cat_features = [x for x in cat_features if x not in impute_missing_features and x not in impute_no_comments_features and x not in drop_features]

In [ ]:
# Create preprocessing pipelines
impute_median_pipeline = Pipeline([
   ('imputer_num', SimpleImputer(strategy='median')),
   ('std_scaler', StandardScaler())
])

impute_missing_pipeline = Pipeline([
   ('imputer_cat', SimpleImputer(strategy='constant', fill_value='missing')),
   ('1hot', OneHotEncoder(handle_unknown='ignore', drop='first'))
])

impute_no_comments_pipeline = Pipeline([
   ('imputer_cat', SimpleImputer(strategy='constant', fill_value='No comments')),
   ('1hot', OneHotEncoder(handle_unknown='ignore', drop='first'))
])

num_pipeline = Pipeline([
   ('std_scaler', StandardScaler())
])

cat_pipeline = Pipeline([
   ('1hot', OneHotEncoder(handle_unknown='ignore', drop='first'))
])

# Create preprocessor
# ColumnTransformer: Any columns not specified in the list of “transformers” are dropped from the dataset by default.
preprocessor = ColumnTransformer([
   ('median', impute_median_pipeline, impute_median_features),
   ('missing', impute_missing_pipeline, impute_missing_features),
   ('nocomment', impute_no_comments_pipeline, impute_no_comments_features),
   ('num', num_pipeline, num_features),
   ('cat', cat_pipeline, cat_features)
])

In [ ]:
# Create second preprocessor for version with dropped package columns
cat_features_wopack = [feature for feature in cat_features if feature not in package_columns]

preprocessor_wopack = ColumnTransformer([
   ('median', impute_median_pipeline, impute_median_features),
   ('missing', impute_missing_pipeline, impute_missing_features),
   ('nocomment', impute_no_comments_pipeline, impute_no_comments_features),
   ('num', num_pipeline, num_features),
   ('cat', cat_pipeline, cat_features_wopack)
])


In [ ]:
pipe_linreg = Pipeline([
   ('preprocessor', preprocessor),
   ('linreg', LinearRegression())
])


## Training a simple Linear Regression model

In [ ]:
# Train model (without log-transformation of target)
pipe_linreg.fit(X_train, y_train)

In [ ]:
# Make predictions
y_pred_test_linreg = pipe_linreg.predict(X_test)
y_pred_train_linreg = pipe_linreg.predict(X_train)

In [ ]:
# Calculate metrics for Linear Regression Model for test data
calculate_metrics(y_test,y_pred_test_linreg, print_metrics=True)

In [ ]:
# Calculate metrics for Linear Regression Model for train data
calculate_metrics(y_train,y_pred_train_linreg, print_metrics=True, calculate_r2=True)

In [ ]:
# Plot actual vs. predicted value for the model
plot_act_vs_pred(y_test, y_pred_test_linreg, 'lin_act_vs_pred_values')

In [ ]:
# Plot the relation of actual value and the difference between actual and predicted value
residual_plot(y_test, y_pred_test_linreg,'lin_residual_plot', y_train, y_pred_train_linreg)

In [ ]:
# Train model again with log-transformation of target
pipe_linreg_log = Pipeline([
   ('preprocessor', preprocessor),
   ('linreg', LinearRegression())
])

pipe_linreg_log.fit(X_train, y_train_log)

In [ ]:
# Make predictions and inverse log-transformation
y_pred_test_linreg_log = pipe_linreg_log.predict(X_test)
y_pred_test_linreg_invlog = log_transformer.inverse_func(y_pred_test_linreg_log)

y_pred_train_linreg_log = pipe_linreg_log.predict(X_train)
y_pred_train_linreg_invlog = log_transformer.inverse_func(y_pred_train_linreg_log)

In [ ]:
# Calculate metrics for Linear Regression Model for test data with log-transformation
calculate_metrics(y_test,y_pred_test_linreg_invlog, print_metrics=True)

Log-transformation of the target-value made the result worse for linear regression!

## Quick overview of performance for different classifiers

In [ ]:
def model_evaluation_log(regressor, X_train, y_train, preprocessor = preprocessor, log=False, version=''):
    if version == 'wopack':
        pipe_regressor = Pipeline([
            ('preprocessor', preprocessor_wopack),
            ('regressor', regressor)
        ])
    else:
        pipe_regressor = Pipeline([
            ('preprocessor', preprocessor),
            ('regressor', regressor)
        ])

    y_pred = cross_val_predict(pipe_regressor, X_train, y_train, cv=10, n_jobs=-1)

    results = {}
    if log:
        y_pred = log_transformer.inverse_func(y_pred)

    results['model'] = regressor
    results['rmse'] = mean_squared_error(y_train, y_pred, squared=False)
    results['mae'] = mean_absolute_error(y_train, y_pred)
    results['version'] = 'normal' + version if not log else 'log' + version
    results['y_pred'] = y_pred
    return results

In [ ]:
# Evaluate different models to make the decision for the best of them to be further adjusted
list_of_estimators = [('lr', LinearRegression()),
                     ('dt', DecisionTreeRegressor(random_state=42)),
                     ('svm', SVR()),
                     ('knn', KNeighborsRegressor()),
                     ('rf', RandomForestRegressor())]

stacking_final_estimator = RandomForestRegressor()

list_of_reg = [LinearRegression(), 
               DecisionTreeRegressor(random_state=42), 
               SVR(), 
               KNeighborsRegressor(), 
               RandomForestRegressor(random_state=42), 
               AdaBoostRegressor(random_state=42), 
               XGBRegressor(random_state=42), 
               VotingRegressor(estimators=list_of_estimators), 
               StackingRegressor(estimators=list_of_estimators, final_estimator=stacking_final_estimator)]#,
               #StackingRegressor(estimators=list_of_estimators, final_estimator=VotingRegressor(estimators=list_of_estimators))]
 
model_results = []
model_results_log = []
model_results_imp = []
model_results_imp_log = []
model_results_wopack = []
model_results_wopack_log = []

for reg in list_of_reg:
   results = model_evaluation_log(reg, X_train, y_train, preprocessor, log=False)
   results_log = model_evaluation_log(reg, X_train, y_train_log, preprocessor, log=True)

   print(results['model'])
   print(f"RSME: {results['rmse']}")
   print(f"MAE: {results['mae']}")
   print(f"RSME log: {results_log['rmse']}")
   print(f"MAE log: {results_log['mae']}")

   results_imp = model_evaluation_log(reg, X_train_imputed_package, y_train, preprocessor, log=False, version='imp')
   results_imp_log = model_evaluation_log(reg, X_train_imputed_package, y_train_log, preprocessor, log=True, version='imp')

   print(f"RSME imp: {results_imp['rmse']}")
   print(f"MAE imp: {results_imp['mae']}")
   print(f"RSME imp log: {results_imp_log['rmse']}")
   print(f"MAE imp log: {results_imp_log['mae']}")
   
   results_wopack = model_evaluation_log(reg, X_train_without_package_cols, y_train, preprocessor, log=False, version='wopack')
   results_wopack_log = model_evaluation_log(reg, X_train_without_package_cols, y_train_log, preprocessor, log=True, version='wopack')

   print(f"RSME wopack: {results_wopack['rmse']}")
   print(f"MAE wopack: {results_wopack['mae']}")
   print(f"RSME wopack log: {results_wopack_log['rmse']}")
   print(f"MAE wopack log: {results_wopack_log['mae']}")

   print("----"*10)

   model_results.append(results)
   model_results_log.append(results_log)

   model_results_imp.append(results_imp)
   model_results_imp_log.append(results_imp_log)

   model_results_wopack.append(results_wopack)
   model_results_wopack_log.append(results_wopack_log)

In [ ]:
# Rank results of the different models by RMSE mean
model_results_df = pd.DataFrame.from_dict(model_results)
model_results_log_df = pd.DataFrame.from_dict(model_results_log)
model_results_imp_df = pd.DataFrame.from_dict(model_results_imp)
model_results_imp_log_df = pd.DataFrame.from_dict(model_results_imp_log)
model_results_wopack_df = pd.DataFrame.from_dict(model_results_wopack)
model_results_wopack_log_df = pd.DataFrame.from_dict(model_results_wopack_log)

model_results_total_df = pd.concat([model_results_df, model_results_log_df, model_results_imp_df, model_results_imp_log_df, model_results_wopack_df, model_results_wopack_log_df], axis=0, ignore_index=True)

model_results_total_df.sort_values('rmse')


All regressors perform better on log-transformed data!

## Plot best model

In [ ]:
# Residual plot AdaBoost (log wopack)
residual_plot(y_train, model_results_total_df.loc[50, 'y_pred'],'1_adaboost_logwopack_residual_plot')

## Train AdaBoost model with GridSearch for log-transformed data without package columns

In [ ]:
# Randomized Grid Search for AdaBoostRegressor
param_grid = {#"regressor__base_estimator": [DecisionTreeRegressor()], # LinearRegression()], KNeighborsRegressor()],
            'regressor__n_estimators': [10, 30, 50, 70, 90],
            'regressor__learning_rate': [0.3, 0.5, 0.7, 1.0],
            'regressor__loss': ['linear', 'square', 'exponential'],
            'regressor__base_estimator__max_depth': np.arange(5,70,5),
            'regressor__base_estimator__min_samples_split': np.arange(5,30, 5),
            'regressor__base_estimator__splitter': ['best', 'random']
            }

dtreg = DecisionTreeRegressor(criterion='squared_error')

pipe_regressor = Pipeline([
        ('preprocessor', preprocessor_wopack),      #preprocessor
        ('regressor', AdaBoostRegressor(base_estimator= dtreg, random_state=42))
    ])

rand_search = RandomizedSearchCV(pipe_regressor, param_distributions=param_grid, scoring='neg_root_mean_squared_error', cv=5, verbose=1, n_jobs=-1, n_iter=100)

In [ ]:
# Train Model with Adaboost, without the package columns and with log-transformed target column
rand_search.fit(X_train_without_package_cols, y_train_log)          #X_train_imputed_package

In [ ]:
print(rand_search.best_score_ * (-1.0))
print(rand_search.best_params_)
#print(gs.best_estimator_)

# save best model
best_model_ada = rand_search.best_estimator_


In [ ]:
y_test_pred_ada_rs_log = best_model_ada.predict(X_test_without_package_cols)
y_test_pred_ada_rs = log_transformer.inverse_func(y_test_pred_ada_rs_log)

y_train_pred_ada_rs_log = best_model_ada.predict(X_train_without_package_cols)
y_train_pred_ada_rs = log_transformer.inverse_func(y_train_pred_ada_rs_log)

In [ ]:
calculate_metrics(y_test, y_test_pred_ada_rs, print_metrics=True)

In [ ]:
calculate_metrics(y_train, y_train_pred_ada_rs, print_metrics=True, calculate_r2=True)

In [ ]:
# Plot actual vs. predicted value for the model
plot_act_vs_pred(y_test, y_test_pred_ada_rs, 'ada_randsearch_act_vs_pred_values')

In [ ]:
# Plot the relation of actual value and the difference between actual and predicted value
residual_plot(y_test, y_test_pred_ada_rs,'ada_randsearch_residual_plot', y_train, y_train_pred_ada_rs)

## Error analysis - Deep dive into difference of high and low total cost

In [ ]:
# Compare travellers with high to low total_cost
train_data_all_df = pd.concat([X_train,y_train], axis=1)

# Divide travellers in those who have high total_cost and those with low total_cost
low_cost_df = train_data_all_df.query('total_cost <= 10000000')  #2818 rows
high_cost_df = train_data_all_df.query('total_cost > 10000000')  #913 rows

no_low = low_cost_df.shape[0]
no_high = high_cost_df.shape[0]

# Create feature groups
# Country not included because there are too many categories in this feature
percentage_grouped_low = low_cost_df.groupby(['main_activity', 'age_group', 'travel_with', 'purpose', 'payment_mode', 'info_source', 'tour_arrangement', 'first_trip_tz', 'country','most_impressing']).count()['ID'] / no_low
percentage_grouped_low = percentage_grouped_low.reset_index()
percentage_grouped_high = high_cost_df.groupby(['main_activity', 'age_group', 'travel_with', 'purpose', 'payment_mode', 'info_source', 'tour_arrangement', 'first_trip_tz', 'country','most_impressing']).count()['ID'] / no_high
percentage_grouped_high = percentage_grouped_high.reset_index()

# join basemodel_train to test data
percentage_total = percentage_grouped_low.merge(percentage_grouped_high, on=['main_activity', 'age_group', 'travel_with', 'purpose', 'payment_mode', 'info_source', 'tour_arrangement', 'first_trip_tz', 'country','most_impressing'], suffixes=('_low', '_high'), how='outer')
percentage_total.rename(columns={'ID_low': 'percentage_low_cost', 'ID_high': 'percentage_high_cost'}, inplace=True)


In [ ]:

percentage_total.sort_values('tour_arrangement').head(30)

#percentage_total.shape[0] # 2285# feature groups

percentage_total.isnull().sum()
# 580 of total groups do not exist for low total_cost
# 1625 of total groups do not exist for high total_cost
# --> not enough data to represent the high total_cost group for some feature-combinations!!

#percentage_total[percentage_total['percentage_high_cost'].notnull()]

#percentage_total.sort_values('percentage_high_cost')

In [ ]:
low_high = [580, 1625, int(percentage_total.shape[0])]
labels = ['missing for trips with low cost', 'missing for trips with high cost', 'Total possible combinations']
low_high

fig, axes = plt.subplots(1,1, figsize=(10, 5))
fig = sns.barplot(labels, low_high)


for p in fig.patches:
    fig.annotate(format(p.get_height(), '.1f'), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center', 
                   size=15,
                   color='Black',
                   xytext = (0, -12), 
                   textcoords = 'offset points')

plt.savefig('images/missing_data_low_vs_high_cost.png')
plt.show()

In [ ]:
low_high = [low_cost_df.shape[0], high_cost_df.shape[0]]
labels = ['Number of trips with low cost', 'Number of trips with high cost']
low_high

fig, axes = plt.subplots(1,1, figsize=(7, 10))
fig = sns.barplot(labels, low_high)

for p in fig.patches:
    fig.annotate(format(p.get_height(), '.1f'), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center', 
                   size=15,
                   color='Black',
                   xytext = (0, -12), 
                   textcoords = 'offset points')

plt.savefig('images/number_trips_low_vs_high_cost.png')
plt.show()


For the group of travellers with high total_cost, it seems that they are only centered in a few groups of feature-combinations.
</br>
The dataset is too small to be assessed by these many features.
</br>
More data is needed!